# Training a Neural Network using Augmentor and Keras

In this notebook, we will train a simple convolutional neural on the MNIST dataset using Augmentor to augment images on the fly using a generator.

## Import Required Libraries

We start by making a number of imports:

In [1]:
import Augmentor

import keras;
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D

import numpy as np

Using Theano backend.
Using gpu device 0: GeForce GTX TITAN X (CNMeM is enabled with initial size: 90.0% of memory, cuDNN not available)


## Define a Convolutional Neural Network

Once the libraries have been imported, we define a small convolutional neural network. See the Keras documentation for details of this network: <https://github.com/fchollet/keras/blob/master/examples/mnist_cnn.py> 

It is a three layer deep neural network, consisting of 2 convolutional layers and a fully connected layer:

In [2]:
batch_size = 128
num_classes = 10
input_shape = (28, 28, 1)

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

Once a network has been defined, you can compile it so that the model is ready to be trained with data:

In [3]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

You can view a summary of the network using the `summary()` function:

In [4]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 12, 12, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 9216)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               1179776   
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
__________

It is important the shape of the input layer matches the batch size that we use later to create our generator, which we shall see later.

## Use Augmentor to Scan Directory for Data

Now we will use Augmentor to scan a directory containing our data that we will eventually feed into the neural network in order to train it. 

When you point a pipeline to a directory, it will scan each subdirectory and treat each subdirectory as a class for your machine learning problem. 

For example, within the directory `mnist`, there are subdirectories for each digit:

```
mnist/
├── 0/
│   ├── 0001.png
│   ├── 0002.png
│   ├── ...
│   └── 5985.png
├── 1/
│   ├── 0001.png
│   ├── 0002.png
│   ├── ...
│   └── 6101.png
├── 2/
│   ├── 0000.png
│   ├── 0001.png
│   ├── ...
│   └── 5801.png
│ ...
├── 9/
│   ├── 0001.png
│   ├── 0002.png
│   ├── ...
│   └── 6001.png
└
```

The directory `0` contains all the images corresponding to the 0 class.

To do this, we instantiate a pipeline object in the `mnist` parent directory:

In [5]:
p = Augmentor.Pipeline("/home/marcus/Documents/mnist/train/")

Initialised with 60000 image(s) found.
Output directory set to /home/marcus/Documents/mnist/train/output.

## Add Operations to the Pipeline

Now that a pipeline object `p` has been created, we can add operations to the pipeline. Below we add several simple  operations:

In [6]:
p.flip_top_bottom(probability=0.1)
p.rotate(probability=0.3, max_left_rotation=5, max_right_rotation=5)

You can view the status of pipeline using the `status()` function, which shows information regarding the number of classes in the pipeline, the number of images, and what operations have been added to the pipeline:

In [7]:
p.status()

Operations: 2
	0: Flip (top_bottom_left_right=TOP_BOTTOM probability=0.1 )
	1: RotateRange (max_right_rotation=5.0 max_left_rotation=-5.0 probability=0.3 )
Images: 60000
Classes: 10
	Class index: 0 Class label: 0 
	Class index: 1 Class label: 1 
	Class index: 2 Class label: 2 
	Class index: 3 Class label: 3 
	Class index: 4 Class label: 4 
	Class index: 5 Class label: 5 
	Class index: 6 Class label: 6 
	Class index: 7 Class label: 7 
	Class index: 8 Class label: 8 
	Class index: 9 Class label: 9 
Dimensions: 1
	Width: 28 Height: 28
Formats: 1
	 PNG

You can remove operations using the appropriate index and the remove_operation(index) function.


## Creating a Generator

A generator will create images indefinitely, and we can use this generator as input into the model created above. The generator is created with a user-defined batch size, which we defined earlier in the `batch_size` variable. This affects the shape of the input layer of your network, so this must match the batch size used to create the generator: 

In [8]:
g = p.keras_generator(batch_size=batch_size)

The generator can now be used to created augmented data. In Python, generators are invoked using their `next()` function - the Augmentor generators will return images indefinitely, and so `next()` can be called as often as required. 

You can view the output of generator manually:

In [9]:
images, labels = g.next()

Images, and their labels, are returned in batches of the size defined above by `batch_size`. The `image_batch` variable is a tuple, containing the augmentented images and their corresponding labels.

To see the label of the first image returned by the generator you can use the array's index:

In [10]:
print(labels[0])

[0 0 0 1 0 0 0 0 0 0]


## Train the Network

We train the network by passing the generator, `g`, to the model's fit function. In Keras, if a generator is used we used the `fit_generator()` function as opposed to the standard `fit()` function. Also, the steps per epoch should roughly equal the total number of images in your dataset divided by the `batch_size`.

Training the network over 5 epochs, we get the following output:

In [11]:
h = model.fit_generator(g, steps_per_epoch=len(p.augmentor_images)/batch_size, epochs=5, verbose=1)

Epoch 1/5
468/468 [==============================] - 33s - loss: 0.5231 - acc: 0.8358    
Epoch 2/5
468/468 [==============================] - 33s - loss: 0.2036 - acc: 0.9382    
Epoch 3/5
468/468 [==============================] - 33s - loss: 0.1610 - acc: 0.9523    
Epoch 4/5
468/468 [==============================] - 33s - loss: 0.1404 - acc: 0.9575    
Epoch 5/5
468/468 [==============================] - 34s - loss: 0.1266 - acc: 0.9616    


## Summary

Using Augmentor with Keras means only that you need to create a generator when you are finished creating your pipeline. This has the advantage that no images need to be saved to disk and are augmented on the fly.